In [70]:
import gzip
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D

In [94]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [95]:
for elem in readGz("/Users/linbinyang/Desktop/course-fall2018/cse258/data/assignment1/train.json.gz"):
    print (elem)
    break

{'reviewTime': '09 26, 2013', 'reviewText': "The model in this picture has them rolled up at the top because they are actually very high waisted! that's my only complaint though, because they are very good quality, and fit really well! I am 5'2&#34; 120lbs with thick thighs and i love them i can't wait to wear them out!", 'helpful': {'nHelpful': 0, 'outOf': 0}, 'reviewerID': 'U490934656', 'reviewHash': 'R798569390', 'categories': [['Clothing, Shoes & Jewelry', 'Women'], ['Clothing, Shoes & Jewelry', 'Novelty, Costumes & More', 'Novelty', 'Clothing', 'Women', 'Leggings']], 'unixReviewTime': 1380153600, 'itemID': 'I402344648', 'rating': 4.0, 'summary': 'High Waisted', 'categoryID': 0}


In [147]:
all_set = []
user_set = set()
item_set = set()
for elem in readGz("/Users/linbinyang/Desktop/course-fall2018/cse258/data/assignment1/train.json.gz"):
    all_set.append(elem)
    user_set.add(elem['reviewerID'])
    item_set.add(elem['itemID'])

In [167]:
# we use cosine similarity measure similarity between users and items
user = defaultdict(list)
item = defaultdict(list)
for elem in all_set:
    item_R = []
    user_R = []
    user_R.append(elem['reviewerID'])
    item_R.append(elem['itemID'])
    user_R.append(elem['rating'])
    item_R.append(elem['rating'])
    user[elem['reviewerID']].append(item_R)
    item[elem['itemID']].append(user_R)

In [171]:
print (f"The total number of users is {len(user)}")

The total number of users is 39239


In [149]:
print (f"The total number of items is {len(item)}")

The total number of items is 19914


In [123]:
def CalculatePetersonSimilarity(user1, user2, user, user_set):
    res = 0
    r_u1_avg = sum([item[1] for item in user[user1]])/len(user[user1])
    r_u2_avg = sum([item[1] for item in user[user2]])/len(user[user2])
    item_intersec = []
    for item_1 in user[user1]:
        for item_2 in user[user2]:
            if item_1[0] == item_2[0]:
                item_intersec.append(item_1)
                break
    if len(item_intersec) == 0:
        return res
    else:
        for elem in item_intersec:
            res = res + (elem[1] - r_u1_avg) * (elem[1] - r_u2_avg)
    deno1 = 0
    for elem in item_intersec:
        deno1 = deno1 + (elem[1] - r_u1_avg)**2
    deno2 = 0
    for elem in item_intersec:
        deno2 = deno2 + (elem[1] - r_u2_avg)**2
    return res/((deno1*deno2)**(1/2))

In [172]:
def CalculateJacardSimilarity_user(user1, user2, user, user_set):
    if user1 not in user or user2 not in user:
        return 0
    item1 = user[user1]
    item2 = user[user2]
    similarity = 0
    for each_item_1 in item1:
        item_id = each_item_1[0]
        for each_item_2 in item2:
            if each_item_2[0] == item_id:
                similarity = similarity + 1
                break
    item_set = set()
    for item in item1:
        item_set.add(item[0])
    for item in item2:
        item_set.add(item[0])
    return similarity/len(item_set)

In [82]:
def CalculateJacardSimilarity_item(item1, item2, item, item_set):
    if item1 not in item_set:
        return 1
    user1 = item[item1]
    user2 = item[item2]
    similarity = 0
    for each_user_1 in user1:
        user_id = each_user_1[0]
        for each_user_2 in user2:
            if each_user_2[0] == user_id:
                similarity = similarity + 1
                break
    user_set = set()
    for user_i in user1:
        user_set.add(user_i[0])
    for user_i in user2:
        user_set.add(user_i[0])
    return similarity/len(user_set)

In [173]:
def BuilderSimilarityUser(user_give, user, user_set):
    res = []
    for user_each in user_set:
        if user_each == user_give:
            continue
        if CalculateJacardSimilarity_user(user_give, user_each, user, user_set) > 0:
            res.append(user_each)
    return res

In [ ]:
user_sim = defaultdict(list)
for any_user in user_set:
    user_sim[any_user] = BuilderSimilarityUser(any_user, user, user_set)

In [ ]:
def Judge(u, i, user_sim, item):
    for each_user in user_sim[u]:
        for each_item in item[each_user]:
            if i == each_item:
                flag = True
                return flag
    return False

In [143]:
def PredictBuy_v3(user_g, item_g, item, user, item_set, user_set):
    if user_g not in user_set or item_g not in item_set:
        return False
    else:
        for each_user in item[item_g]:
            if CalculateJacardSimilarity_user(user_g, each_user[0], user, user_set) > 0.2:
                return True
    return False

In [117]:
print (CalculatePetersonSimilarity('U507366950', 'U507366950', user, user_set))

[['I464613034', 5.0], ['I872967861', 5.0], ['I506949867', 5.0], ['I723155560', 5.0], ['I029346709', 5.0], ['I264463508', 2.0], ['I075704956', 5.0], ['I847796665', 5.0], ['I787056386', 2.0], ['I225955774', 2.0], ['I323568513', 5.0], ['I241584112', 5.0], ['I930339650', 3.0]]
1.0


In [144]:
f = open("/Users/linbinyang/Downloads/assignment1/pairs_Purchase.txt")
f_1 = open("output_pp20.txt","w")
index = 0
while 1:
    line = f.readline().strip()
    if not line:
        break
    if index == 0:
        f_1.write(line+'\n')
        index = index + 1
        continue
    u = line.split("-")[0]
    i = line.split("-")[1]
    flag = Judge(u, i, user_sim, item)
    if flag:
        f_1.write(u + '-' + i + ',' + '1'+'\n')
    else:
        f_1.write(u + '-' + i + ',' + '0'+'\n')
f_1.close()
f.close()